ChatGPT Chat
https://chat.openai.com/share/492f45ea-5d07-4c17-a351-fc32b7b16f87

In [1]:
import pandas as pd

In [2]:
# Memuat data CSV ke dalam DataFrame
df = pd.read_csv('ecommerce-session-bigquery.csv')

# Menampilkan beberapa baris pertama DataFrame
df.head()

,fullVisitorId,channelGrouping,time,country,city,totalTransactionRevenue,transactions,timeOnSite,pageviews,sessionQualityDim,...,itemQuantity,itemRevenue,transactionRevenue,transactionId,pageTitle,searchKeyword,pagePathLevel1,eCommerceAction_type,eCommerceAction_step,eCommerceAction_option
0,2515546493837534633,Organic Search,966564,Taiwan,(not set),NaN,NaN,1567.0,82.0,17.0,...,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
1,9361741997835388618,Organic Search,157377,France,not available in demo dataset,NaN,NaN,321.0,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
2,7313828956068851679,Referral,228279,United States,San Francisco,NaN,NaN,927.0,11.0,63.0,...,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
3,6036794406403793540,Organic Search,1615618,United States,Boulder,NaN,NaN,1616.0,13.0,38.0,...,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN
4,7847280609739507227,Organic Search,37832,Canada,not available in demo dataset,NaN,NaN,1222.0,45.0,53.0,...,NaN,NaN,NaN,NaN,NaN,NaN,/storeitem.html,0,1,NaN


In [7]:
# Mengonversi kolom 'date' menjadi format datetime
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

# Memfilter baris dengan transactionRevenue tidak null
df_filtered = df[df['totalTransactionRevenue'].notnull()]

# Mengelompokkan berdasarkan tanggal dan produk, menjumlahkan pendapatan transaksi
top_products = df_filtered.groupby(['date', 'v2ProductName'])['totalTransactionRevenue'].sum().reset_index()

# Menemukan produk teratas untuk setiap tanggal
top_products = top_products.loc[top_products.groupby('date')['totalTransactionRevenue'].idxmax()]

# Menampilkan produk teratas per hari berdasarkan pendapatan transaksi total
top_products

,date,v2ProductName,totalTransactionRevenue
0,2016-08-01,Chevron Shopper,1.447900e+08
4,2016-08-02,Chevron Shopper,1.912000e+08
6,2016-08-04,Google Lunch Bag,1.138500e+08
7,2016-08-05,Chevron Shopper,2.647800e+08
15,2016-08-06,Waterpoof Gear Bag,1.157700e+09
...,...,...,...
409,2017-07-12,Google Leather Perforated Journal,1.549800e+08
410,2017-07-15,Google Men's Zip Hoodie,1.233900e+08
411,2017-07-22,Google Leather Perforated Journal,2.730000e+08
412,2017-07-26,Metal Texture Roller Pen,6.757000e+07


In [8]:
# Mengelompokkan berdasarkan produk dan tanggal, menghitung jumlah transaksi
transactions_per_product = df.groupby(['v2ProductName', 'date'])['transactions'].sum().reset_index()

# Menghitung skor z untuk transaksi untuk mendeteksi anomali
transactions_per_product['z_score'] = (transactions_per_product.groupby('v2ProductName')['transactions']
                                       .transform(lambda x: (x - x.mean()) / x.std()))

# Mendeteksi anomali (dengan menganggap skor z > 3 sebagai anomali)
anomalies = transactions_per_product[transactions_per_product['z_score'] > 3]

# Menampilkan anomali yang terdeteksi
anomalies

,v2ProductName,date,transactions,z_score
17,1 oz Hand Sanitizer,2016-09-16,5.0,4.310075
123,Collapsible Shopping Bag,2016-08-26,7.0,4.530307
214,Electronics Accessory Pouch,2016-08-17,4.0,3.618694
391,Google Kick Ball,2016-09-16,3.0,3.469443
416,Google Laptop and Cell Phone Stickers,2016-08-07,1.0,3.614784
532,Google Pet Feeding Mat,2016-10-02,2.0,3.269092
552,Google Sunglasses,2016-09-16,4.0,3.392615
588,Google Tote Bag,2016-08-26,6.0,4.170352
591,Google Tote Bag,2016-08-29,5.0,3.364555
666,Google Zipper-front Sports Bag,2016-09-01,4.0,3.522875


In [18]:
# Mengelompokkan berdasarkan kota dan menjumlahkan pendapatan transaksi
city_revenue = df_filtered.groupby('city')['totalTransactionRevenue'].sum().reset_index()

# Menampilkan kota paling menguntungkan
city_revenue.sort_values(by="totalTransactionRevenue", ascending=False)


,city,totalTransactionRevenue
29,not available in demo dataset,6.510244e+10
14,New York,2.125081e+10
21,San Francisco,7.774340e+09
13,Mountain View,5.713290e+09
27,Toronto,5.427980e+09
7,Chicago,4.621840e+09
1,Ann Arbor,3.898660e+09
2,Atlanta,3.182210e+09
12,Maracaibo,2.853540e+09
11,Los Angeles,1.674930e+09
